In [27]:
import numpy as np
from scipy.misc import imread,imresize
import os
from os import listdir
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

img_size = 150
grayscale_images = True
num_class = 10
test_size = 0.2


def get_img(data_path):
    # Getting image array from path:
    img = imread(data_path, flatten=grayscale_images)
    img = imresize(img, (img_size, img_size, 1 if grayscale_images else 3))
    return img

def get_dataset(dataset_path='/home/krithika/Desktop/Sign Language'):
    # Getting all data from data path:
    try:
        X = np.load('Desktop/Sign Language/npy_dataset/X.npy')
        Y = np.load('Desktop/Sign Language/npy_dataset/Y.npy')
    except:
        labels = listdir(dataset_path) # Geting labels
        X = []
        Y = []
        for i, label in enumerate(labels):
            datas_path = dataset_path+'/'+label
            for data in listdir(datas_path):
                img = get_img(datas_path+'/'+data)
                X.append(img)
                Y.append(i)
        # Create dateset:
        X = 1-np.array(X).astype('float32')/255.
        Y = np.array(Y).astype('float32')
        Y = to_categorical(Y, num_class)
        np.save('Desktop/Sign Language/npy_dataset/X.npy', X)
        np.save('Desktop/Sign Language/npy_dataset/Y.npy', Y)
    X, X_test, Y, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
    return X, X_test, Y, Y_test

if __name__ == '__main__':
    X_train,X_test,Y_train,Y_test=get_dataset()

In [32]:
print(X_train.shape)
X_train=np.array(X_train).reshape(-1,64,64,1)
X_test=np.array(X_test).reshape(-1,64,64,1)

model = Sequential()
model.add(Convolution2D(32,3,3, border_mode ="same", input_shape=(64, 64, 1)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64,2,2, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), dim_ordering='th'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='model.hdf5', 
                               verbose=1, save_best_only=True)
hist = model.fit(X_train,Y_train, batch_size=128, epochs=20,
          validation_split=0.2, callbacks=[checkpointer],
          verbose=1, shuffle=True)

model.load_weights('model.hdf5')
# evaluate test accuracy
score = model.evaluate(X_test, Y_test, verbose=0)

acc=100*score[1]

print('Test accuracy: %.2f%%' % acc)

(1649, 64, 64, 1)


/home/krithika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 1..., padding="same")`
  del sys.path[0]
/home/krithika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
/home/krithika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


Train on 1319 samples, validate on 330 samples
Epoch 1/20
1319/1319 [==============================] - 5s 4ms/step - loss: 3.4159 - acc: 0.1016 - val_loss: 2.2846 - val_acc: 0.1545

Epoch 00001: val_loss improved from inf to 2.28462, saving model to model.hdf5
Epoch 2/20
1319/1319 [==============================] - 5s 4ms/step - loss: 2.2624 - acc: 0.1698 - val_loss: 2.2212 - val_acc: 0.2182

Epoch 00002: val_loss improved from 2.28462 to 2.22120, saving model to model.hdf5
Epoch 3/20
1319/1319 [==============================] - 5s 4ms/step - loss: 2.1380 - acc: 0.3161 - val_loss: 2.0304 - val_acc: 0.3152

Epoch 00003: val_loss improved from 2.22120 to 2.03038, saving model to model.hdf5
Epoch 4/20
1319/1319 [==============================] - 5s 4ms/step - loss: 1.9136 - acc: 0.3867 - val_loss: 1.7597 - val_acc: 0.4545

Epoch 00004: val_loss improved from 2.03038 to 1.75969, saving model to model.hdf5
Epoch 5/20
1319/1319 [==============================] - 5s 4ms/step - loss: 1.6585 - 